In [2]:
import numpy as np
import math
import csv

class Node:
    def __init__(self,attribute):
        self.attribute=attribute
        self.children=[]
        self.answer=None

  

In [3]:
def read_data(filename):
        with open(filename,'r')as csvfile:
            datareader=csv.reader(csvfile,delimiter=',')
            headers=next(datareader)
            traindata=[row for row in datareader]
        return headers,traindata

In [4]:
def read_test_data(filename):
        with open(filename,'r')as csvfile:
            datareader=csv.reader(csvfile,delimiter=',')
            headers=next(datareader)
            testdata=[row for row in datareader]
        return testdata

In [5]:
def subtables(data,col,delete):
    dict={}
    items=np.unique(data[:,col])
    for item in items:
        dict[item]=data[data[:,col]==item]
        if delete:
            dict[item]=np.delete(dict[item],col,axis=1)
    return items,dict





In [6]:
import numpy as np

def entropy(S):
    items, counts = np.unique(S, return_counts=True)
    probabilities = counts / len(S)  # Calculate probabilities correctly
    return -np.sum(probabilities * np.log2(probabilities))  # Calculate entropy


In [7]:
import numpy as np
import math

def entropy(S):
    items, counts = np.unique(S, return_counts=True)
    probabilities = counts / len(S)
    return -np.sum(probabilities * np.log2(probabilities))

def gain_ratio(data, col):
    total_entropy = entropy(data[:, -1])
    items, dictionary = subtables(data, col, delete=False)  # Assuming subtables is a function you have defined elsewhere
    
    subset_entropy = 0
    intrinsic_value = 0
    total_size = data.shape[0]
    
    for item in items:
        subset = dictionary[item]
        ratio = len(subset) / total_size
        subset_entropy += ratio * entropy(subset[:, -1])  # Call entropy for each subset
        intrinsic_value -= ratio * math.log2(ratio) if ratio != 0 else 0  # Added a check for zero ratio
    
    if intrinsic_value == 0:
        return 0
    
    information_gain = total_entropy - subset_entropy
    return information_gain / intrinsic_value


In [8]:
def create_node(data,metadata):
    if len(np.unique(data[:,-1]))==1:
        node=Node(None)
        node.answer=np.unique(data[:,-1])[0]
        return node
    gains=[gain_ratio(data,col)for col in range(data.shape[1]-1)]
    split=np.argmax(gains)
    node=Node(metadata[split])
    items,dict=subtables(data,split,delete=True)
    for item in items:
        child=create_node(dict[item],np.delete(metadata,split))
        node.children.append((item,child))
    return node

In [9]:
def predict(node,instance,metadata):
    if node.answer is not None:
        return node.answer
    value=instance[metadata.index(node.attribute)]
    for item,child in node.children:
        if item==value:
            return predict(child,instance,metadata)

In [10]:
def empty(size):
    return" "*size
def print_tree(node,level):
    if node.answer is not None:
        print(empty(level),node.answer)
        return
    print(empty(level),node.attribute)
    for value,n in node.children:
        print(empty(level +1),value)
        print_tree(n,level+2)

In [11]:
def print_predictions(node,test_data,metadata):
    test_instances=test_data[1:]
    for instance in test_instances:
        prediction=predict(node,instance,metadata)
        print(f"The test_instance:{instance}")
        print(f"The predicted label:{prediction}\n")

In [12]:
metadata,traindata=read_data("train.csv")
test_data=read_test_data("test.csv")
data=np.array(traindata)
node=create_node(data,metadata)
print("Decision Tree Structure:")
print_tree(node,0)
print("/nPrediction for test data:")
print_predictions(node,test_data,metadata)

Decision Tree Structure:
 weather
  cloudy
   yes
  rain
   wind
    strong
     no
    weak
     yes
  sunny
   humidity
    high
     no
    normal
     yes
/nPrediction for test data:
The test_instance:['sunny', 'hot', 'high', 'strong']
The predicted label:no

The test_instance:['cloudy', 'hot', 'high', 'weak']
The predicted label:yes



In [13]:
import numpy as np
import math
import csv
